<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/work/stock/prepare_data_momentum_algo_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_ta --q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [2]:
from statsmodels.regression.rolling import RollingOLS
#import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

In [3]:
def get_nifty50_tickers():
    #data = pd.read_csv('https://raw.githubusercontent.com/anirbanghoshsbi/data/main/STOCKS100.txt', sep='\t')
    nifty50_url = "https://raw.githubusercontent.com/anirbanghoshsbi/data/main/ind_nifty50list.csv"
    nifty50 = pd.read_csv(nifty50_url)
    tickers = nifty50['Symbol'].apply(lambda x: x + ".NS").tolist()
    #reject_lst=['MOTHERSUMI.NS', 'CADILAHC.NS', 'ADANITRANS.NS', 'SRTRANSFIN.NS', 'INFRATEL-EQ.NS','HDFC.NS','LTI.NS']
    #nifty_final= [item for item in nifty50 if item not in reject_lst]

    return tickers


In [4]:
symbols_list = get_nifty50_tickers()

end_date = '2023-08-27'

start_date = '2020-01-02'
df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

df.index.names = ['date', 'ticker']

df.columns = df.columns.str.lower()



[*********************100%%**********************]  50 of 50 completed


In [5]:
df

Price                       adj close        close         high          low  \
date       ticker                                                              
2020-01-02 ADANIENT.NS     209.144470   211.199997   213.199997   207.500000   
           ADANIPORTS.NS   369.654572   383.149994   384.700012   376.500000   
           APOLLOHOSP.NS  1473.526978  1494.650024  1509.199951  1428.000000   
           ASIANPAINT.NS  1724.335205  1790.650024  1799.900024  1783.400024   
           AXISBANK.NS     753.666260   756.950012   759.000000   747.599976   
...                               ...          ...          ...          ...   
2023-08-25 TECHM.NS       1155.407593  1189.500000  1197.199951  1178.449951   
           TITAN.NS       3057.196533  3067.199951  3074.750000  3037.550049   
           ULTRACEMCO.NS  8023.866211  8071.450195  8190.000000  8061.000000   
           UPL.NS          582.148438   583.200012   588.750000   580.000000   
           WIPRO.NS        408.828125   409.700012   411.299988   407.100006   

Price                            open   volume  
date       ticker                               
2020-01-02 ADANIENT.NS     208.000000  2991937  
           ADANIPORTS.NS   377.950012  5919760  
           APOLLOHOSP.NS  1428.000000  2000143  
           ASIANPAINT.NS  1791.000000   411266  
           AXISBANK.NS     750.000000  5156046  
...                               ...      ...  
2023-08-25 TECHM.NS       1190.000000  1213340  
           TITAN.NS       3055.000000   648003  
           ULTRACEMCO.NS  8179.000000   191154  
           UPL.NS          585.000000  1132264  
           WIPRO.NS        410.000000  3299255  

[45350 rows x 6 columns]

In [6]:
df['garman_klass_vol'] = ((np.log(df['high'])-np.log(df['low']))**2)/2-(2*np.log(2)-1)*((np.log(df['adj close'])-np.log(df['open']))**2)

df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,0])

df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,1])

df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:,2])

def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'],
                        low=stock_data['low'],
                        close=stock_data['close'],
                        length=14)
    return atr.sub(atr.mean()).div(atr.std())

df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

df['dollar_volume'] = (df['adj close']*df['volume'])/1e6

df

Price                       adj close        close         high          low  \
date       ticker                                                              
2020-01-02 ADANIENT.NS     209.144470   211.199997   213.199997   207.500000   
           ADANIPORTS.NS   369.654572   383.149994   384.700012   376.500000   
           APOLLOHOSP.NS  1473.526978  1494.650024  1509.199951  1428.000000   
           ASIANPAINT.NS  1724.335205  1790.650024  1799.900024  1783.400024   
           AXISBANK.NS     753.666260   756.950012   759.000000   747.599976   
...                               ...          ...          ...          ...   
2023-08-25 TECHM.NS       1155.407593  1189.500000  1197.199951  1178.449951   
           TITAN.NS       3057.196533  3067.199951  3074.750000  3037.550049   
           ULTRACEMCO.NS  8023.866211  8071.450195  8190.000000  8061.000000   
           UPL.NS          582.148438   583.200012   588.750000   580.000000   
           WIPRO.NS        408.828125   409.700012   411.299988   407.100006   

Price                            open   volume  garman_klass_vol        rsi  \
date       ticker                                                             
2020-01-02 ADANIENT.NS     208.000000  2991937          0.000356        NaN   
           ADANIPORTS.NS   377.950012  5919760          0.000042        NaN   
           APOLLOHOSP.NS  1428.000000  2000143          0.001149        NaN   
           ASIANPAINT.NS  1791.000000   411266         -0.000513        NaN   
           AXISBANK.NS     750.000000  5156046          0.000105        NaN   
...                               ...      ...               ...        ...   
2023-08-25 TECHM.NS       1190.000000  1213340         -0.000212  53.874587   
           TITAN.NS       3055.000000   648003          0.000074  59.615981   
           ULTRACEMCO.NS  8179.000000   191154         -0.000016  45.607143   
           UPL.NS          585.000000  1132264          0.000103  32.808704   
           WIPRO.NS        410.000000  3299255          0.000050  52.125909   

Price                       bb_low    bb_mid   bb_high       atr      macd  \
date       ticker                                                            
2020-01-02 ADANIENT.NS         NaN       NaN       NaN       NaN       NaN   
           ADANIPORTS.NS       NaN       NaN       NaN       NaN       NaN   
           APOLLOHOSP.NS       NaN       NaN       NaN       NaN       NaN   
           ASIANPAINT.NS       NaN       NaN       NaN       NaN       NaN   
           AXISBANK.NS         NaN       NaN       NaN       NaN       NaN   
...                            ...       ...       ...       ...       ...   
2023-08-25 TECHM.NS       6.983704  7.052852  7.122000 -0.498022  0.462247   
           TITAN.NS       7.964734  8.004246  8.043759 -0.106071  0.241465   
           ULTRACEMCO.NS  8.986518  9.004615  9.022711 -0.791995 -0.371010   
           UPL.NS         6.351733  6.393435  6.435137 -1.971558 -0.827105   
           WIPRO.NS       5.990814  6.020179  6.049544 -1.047694  0.177159   

Price                     dollar_volume  
date       ticker                        
2020-01-02 ADANIENT.NS       625.747079  
           ADANIPORTS.NS    2188.266346  
           APOLLOHOSP.NS    2947.264669  
           ASIANPAINT.NS     709.160442  
           AXISBANK.NS      3885.937904  
...                                 ...  
2023-08-25 TECHM.NS         1401.902249  
           TITAN.NS         1981.072525  
           ULTRACEMCO.NS    1533.794122  
           UPL.NS            659.145718  
           WIPRO.NS         1348.828236  

[45350 rows x 14 columns]

In [7]:
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open',
                                                          'high', 'low', 'close']]

data = (pd.concat([df.unstack('ticker')['dollar_volume'].resample('W').mean().stack('ticker').to_frame('dollar_volume'),
                   df.unstack()[last_cols].resample('W').last().stack('ticker')],
                  axis=1)).dropna()

data

dollar_volume    adj close  garman_klass_vol  \
date       ticker                                                        
2020-02-09 ADANIENT.NS      1284.348557   237.169052          0.000414   
           ADANIPORTS.NS    1120.899366   356.533600         -0.000393   
           APOLLOHOSP.NS     826.832412  1639.645752          0.000452   
           ASIANPAINT.NS    2760.819736  1789.865112         -0.000424   
           AXISBANK.NS      6831.266553   744.904419          0.000163   
...                                 ...          ...               ...   
2023-08-27 TECHM.NS         1445.271722  1155.407593         -0.000212   
           TITAN.NS         2013.787435  3057.196533          0.000074   
           ULTRACEMCO.NS    1761.491410  8023.866211         -0.000016   
           UPL.NS            830.322449   582.148438          0.000103   
           WIPRO.NS         1329.591394   408.828125          0.000050   

                                rsi    bb_low    bb_mid   bb_high       atr  \
date       ticker                                                             
2020-02-09 ADANIENT.NS    68.518072  5.358003  5.419424  5.480845 -1.073364   
           ADANIPORTS.NS  40.422045  5.863052  5.906052  5.949052 -1.515079   
           APOLLOHOSP.NS  63.258033  7.303794  7.384677  7.465559 -1.676813   
           ASIANPAINT.NS  59.294253  7.432916  7.470514  7.508113 -1.797676   
           AXISBANK.NS    54.419511  6.561095  6.592175  6.623255 -0.623344   
...                             ...       ...       ...       ...       ...   
2023-08-27 TECHM.NS       53.874587  6.983704  7.052852  7.122000 -0.498022   
           TITAN.NS       59.615981  7.964734  8.004246  8.043759 -0.106071   
           ULTRACEMCO.NS  45.607143  8.986518  9.004615  9.022711 -0.791995   
           UPL.NS         32.808704  6.351733  6.393435  6.435137 -1.971558   
           WIPRO.NS       52.125909  5.990814  6.020179  6.049544 -1.047694   

                              macd  
date       ticker                   
2020-02-09 ADANIENT.NS   -0.103680  
           ADANIPORTS.NS -0.543256  
           APOLLOHOSP.NS  0.327105  
           ASIANPAINT.NS  0.313570  
           AXISBANK.NS   -0.229486  
...                            ...  
2023-08-27 TECHM.NS       0.462247  
           TITAN.NS       0.241465  
           ULTRACEMCO.NS -0.371010  
           UPL.NS        -0.827105  
           WIPRO.NS       0.177159  

[9300 rows x 9 columns]

In [8]:
def calculate_returns(df):

    outlier_cutoff = 0.005

    lags = [1, 2, 3, 6, 9, 12]

    for lag in lags:

        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df


data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

data

dollar_volume    adj close  garman_klass_vol  \
date       ticker                                                        
2020-05-03 ADANIENT.NS       638.173232   141.048111          0.001232   
           ADANIPORTS.NS     857.830016   282.975189          0.000029   
           APOLLOHOSP.NS    1446.710284  1377.134399         -0.000069   
           ASIANPAINT.NS    4141.892443  1700.372803         -0.000632   
           AXISBANK.NS     20002.733128   442.969940          0.001011   
...                                 ...          ...               ...   
2023-08-27 TECHM.NS         1445.271722  1155.407593         -0.000212   
           TITAN.NS         2013.787435  3057.196533          0.000074   
           ULTRACEMCO.NS    1761.491410  8023.866211         -0.000016   
           UPL.NS            830.322449   582.148438          0.000103   
           WIPRO.NS         1329.591394   408.828125          0.000050   

                                rsi    bb_low    bb_mid   bb_high       atr  \
date       ticker                                                             
2020-05-03 ADANIENT.NS    42.098465  4.865775  4.937335  5.008894 -1.059009   
           ADANIPORTS.NS  53.096811  5.457498  5.555381  5.653264 -0.901388   
           APOLLOHOSP.NS  49.914658  6.991098  7.165275  7.339451 -0.549708   
           ASIANPAINT.NS  51.370349  7.305753  7.412304  7.518856  0.662143   
           AXISBANK.NS    45.952190  5.840863  6.019762  6.198661  2.689953   
...                             ...       ...       ...       ...       ...   
2023-08-27 TECHM.NS       53.874587  6.983704  7.052852  7.122000 -0.498022   
           TITAN.NS       59.615981  7.964734  8.004246  8.043759 -0.106071   
           ULTRACEMCO.NS  45.607143  8.986518  9.004615  9.022711 -0.791995   
           UPL.NS         32.808704  6.351733  6.393435  6.435137 -1.971558   
           WIPRO.NS       52.125909  5.990814  6.020179  6.049544 -1.047694   

                              macd  return_1m  return_2m  return_3m  \
date       ticker                                                     
2020-05-03 ADANIENT.NS   -0.249541   0.050130  -0.008725  -0.002463   
           ADANIPORTS.NS -0.193158   0.071864   0.040124   0.048190   
           APOLLOHOSP.NS -0.249919   0.025634   0.007342   0.028974   
           ASIANPAINT.NS  0.287729  -0.037673   0.000697   0.021216   
           AXISBANK.NS   -0.825093   0.101374  -0.036051   0.019262   
...                            ...        ...        ...        ...   
2023-08-27 TECHM.NS       0.462247  -0.017592  -0.012761   0.004040   
           TITAN.NS       0.241465   0.005491   0.008243   0.018346   
           ULTRACEMCO.NS -0.371010  -0.017456  -0.003788  -0.003555   
           UPL.NS        -0.827105   0.008212  -0.013853  -0.011259   
           WIPRO.NS       0.177159  -0.004858  -0.004063   0.000693   

                          return_6m  return_9m  return_12m  
date       ticker                                           
2020-05-03 ADANIENT.NS     0.005650  -0.046505   -0.042382  
           ADANIPORTS.NS   0.020794  -0.016985   -0.019072  
           APOLLOHOSP.NS   0.003803  -0.024044   -0.014434  
           ASIANPAINT.NS   0.001500  -0.002005   -0.004265  
           AXISBANK.NS     0.006417  -0.048704   -0.042388  
...                             ...        ...         ...  
2023-08-27 TECHM.NS       -0.001085   0.010062    0.006967  
           TITAN.NS        0.000634   0.005491    0.006354  
           ULTRACEMCO.NS  -0.001517   0.000386    0.002695  
           UPL.NS         -0.012579  -0.012998   -0.011780  
           WIPRO.NS        0.001904   0.008411    0.000972  

[8700 rows x 15 columns]

In [9]:
data.to_csv('nifty_50_weekly_data.csv')